In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time_series.time_series_models import KernelRidgeRegression
from time_series.kernels import GaussianKernel

2025-11-17 17:32:47.876 | INFO     | time_series.config:<module>:13 - PROJ_ROOT path is: /home/james/Repo/PhD Repo/time_series_clustering


In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator
from sklearn.metrics import silhouette_score, rand_score, adjusted_rand_score
from sklearn.cluster import KMeans

In [3]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform

In [4]:
import itertools
import tqdm

In [5]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

/home/james/Repo/PhD Repo/time_series_clustering/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class DataHolder:
    def __init__(self, **kwargs):
        self.update(**kwargs)
    
    def update(self, **kwargs):
        self.__dict__.update(kwargs)

In [7]:
class OptunaHyperparameterTuning:
    def __init__(self, estimator, param_grid, scoring=None, nfolds=5, n_trials=30):
        self.estimator = estimator
        self.param_grid = param_grid
        self.scoring = scoring if scoring else mean_squared_error
        self.nfolds = nfolds
        self.n_trials = n_trials
    
    def fit(self, X, y=None):
        def objective(trial):
            parameters = dict()

            for p_name, p_params in self.param_grid.items():
                if p_params["type"] == "float":
                    step = p_params["step"] if "step" in p_params else None
                    parameters[p_name] = trial.suggest_float(p_name, p_params["min"], p_params["max"], step=step)
            
            model = self.estimator(**parameters)

            kf = KFold(n_splits=self.nfolds, random_state=None, shuffle=False)
            scores = []
            
            for i, (train_index, test_index) in enumerate(kf.split(X)):
                X_train = X[train_index]
                X_test = X[test_index]

                y_train = y[train_index]
                y_test = y[test_index] 

                model.fit(X_train, y_train)

                y_pred = model.predict(X_test)

                score = self.scoring(y_test, y_pred)
                scores.append(score)
            
            cv_score = np.mean(scores)
            return cv_score


        study = optuna.create_study()
        study.optimize(
            objective,
            n_trials=self.n_trials,
        )

        self.best_params = study.best_params

In [8]:
class GridSearch:
    def __init__(self, estimator, param_grid, scoring=None, nfolds=5):
        self.estimator = estimator
        self.param_grid = param_grid
        self.scoring = scoring if scoring else mean_squared_error
        self.nfolds = nfolds
    
    def fit(self, X, y=None):
        param_names = list(self.param_grid.keys())
        param_values = list(itertools.product(*self.param_grid.values()))

        result = []

        best_score = np.inf
        best_params = []
        
        for p in param_values:
            mapping = {param_names[i]:p[i] for i in range(len(param_names))}
            model = self.estimator(**mapping)

            kf = KFold(n_splits=self.nfolds, random_state=None, shuffle=False)
            scores = []
            
            for i, (train_index, test_index) in enumerate(kf.split(X)):
                X_train = X[train_index]
                X_test = X[test_index]

                y_train = y[train_index]
                y_test = y[test_index] 

                model.fit(X_train, y_train)

                y_pred = model.predict(X_test)

                score = self.scoring(y_test, y_pred)
                scores.append(score)
            
            cv_score = np.mean(scores)
            result.append(
                (mapping, cv_score)
            )

            if cv_score < best_score:
                best_score = cv_score
                best_params = p
        
        self.best_score = best_score
        self.best_params = best_params

        self.cv_results = result


In [9]:
class KRRWrapper(BaseEstimator):
    def __init__(self, bandwidth, reg):
        self.model = KernelRidgeRegression(
            kernels=[GaussianKernel(bandwidth=bandwidth)],
            reg = reg
        )

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

In [10]:
def generate_time_series(x0:list, f, N=200, epsilon=0):
    x = np.zeros(shape=(N))
    for i in range(len(x0)):
        x[i] = x0[i]

    for i in range(len(x0), N):
        x[i] = f(x[:i]) + np.random.normal(0, epsilon)

    return x

In [ ]:
dynamics = [
    lambda x: np.sin(1*x[-1]),
    lambda x: np.sin(2*x[-1]),
    lambda x: np.sin(4*x[-1]),
]

paramsearch = OptunaHyperparameterTuning(
    KRRWrapper,
    param_grid={
        "bandwidth": dict(type="float", min=1e-9, max=4),
        "reg": dict(type="float", min=1e-12, max=1e-2)
    },
    scoring=mean_squared_error,
    n_trials=100
)

In [ ]:
epsilon = 1
n_sets = 10
lag = 1

datasets = []

for d_id, fd in enumerate(dynamics):
    for i in range(n_sets):
        x0 = np.random.random()*2
        d = generate_time_series(x0, fd, 300, epsilon=epsilon)
        datasets.append(
            DataHolder(
                data=d,
                label=d_id
            )
        )

In [15]:
for d in tqdm.tqdm(datasets):
    # N = len(d.data) - 1
    # x_train, y_train, x_test, y_test = d.data[:-1][:N//2], d.data[1:][:N//2], d.data[:-1][N//2:], d.data[1:][N//2:]
    x_train, y_train = d.data[:-1], d.data[1:]

    paramsearch.fit(x_train, y_train)
    bandwidth = paramsearch.best_params["bandwidth"]
    reg = paramsearch.best_params["reg"]

    model = KernelRidgeRegression(
        kernels=[GaussianKernel(bandwidth=bandwidth)],
        reg = reg,
        lag=lag
    )

    model.fit(x_train)

    # print(mean_squared_error(model.predict(x_test), y_test[1:]))
    
    d.update(
        bandwidth=bandwidth,
        model=model,
        kernel=model.kernels[0],
        alpha=model.alpha
    )

100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


In [ ]:
# Clustering 1
similarity_matrix = np.zeros((len(datasets), len(datasets)))
distance_matrix = np.zeros((len(datasets), len(datasets)))

kernel = GaussianKernel(bandwidth=1)

for i in range(len(datasets)):
    dii = (datasets[i].alpha.T@kernel(datasets[i].data[:-lag-1]/datasets[i].bandwidth, datasets[i].data[:-lag-1]/datasets[i].bandwidth)@datasets[i].alpha)[0][0]
    for j in range(i, len(datasets)):
        djj = (datasets[j].alpha.T@kernel(datasets[j].data[:-lag-1]/datasets[j].bandwidth, datasets[j].data[:-lag-1]/datasets[j].bandwidth)@datasets[j].alpha)[0][0]

        dij = (datasets[i].alpha.T@kernel(datasets[i].data[:-lag-1]/datasets[i].bandwidth, datasets[j].data[:-lag-1]/datasets[j].bandwidth)@datasets[j].alpha)[0][0]

        similarity_matrix[i][j] = dij
        distance_matrix[i][j] = dii + djj - 2*dij
        # similarity_matrix[i][j] = dij/np.sqrt(dii*djj) # Cosine distance
        similarity_matrix[j][i] = similarity_matrix[i][j]
        distance_matrix[j][i] = distance_matrix[i][j]

In [ ]:
# Graph laplacian
D = np.diag(similarity_matrix.sum(axis=0))

L = D - similarity_matrix

evals, evecs = np.linalg.eig(L)
eval_idx = np.argsort(evals)

evals = evals[eval_idx]
evecs = evecs[:, eval_idx]

kmeans = KMeans(n_clusters=3)
elbow_idx = (np.where(evals[1:] - evals[:-1] > 1e-6)[0] + 1)[1] # Find the first instance where the increase in eval size is less than 1e-6. Call this the elbow
labels = kmeans.fit_predict(np.abs(evecs[:, :elbow_idx]))

cluster_labels = pd.Series(labels)

for c in cluster_labels.unique():
    cluster_indices = cluster_labels[cluster_labels == c].index
    cluster_truth_labels = [datasets[i].label for i in cluster_indices]

    print(f"{c}: {cluster_truth_labels}")

print(adjusted_rand_score(
    labels_true=[d.label for d in datasets],
    labels_pred=labels
), rand_score(
    labels_true=[d.label for d in datasets],
    labels_pred=labels  
))

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

for n in range(len(dynamics)):
    for i in range(4):
        for j in range(4):
            ax[i, j].scatter(evecs[n*n_sets:(n+1)*n_sets, i], evecs[n*n_sets:(n+1)*n_sets, j])

In [ ]:
plt.imshow(similarity_matrix)

In [ ]:
# distance_matrix[distance_matrix < 0] = 0

# Assuming D is your full distance matrix
condensed_D = squareform(distance_matrix)  # if D is a full matrix
Z = linkage(condensed_D, method='average')
labels = fcluster(Z, t=len(dynamics), criterion='maxclust')  # t is number of clusters

cluster_labels = pd.Series(labels)

for c in cluster_labels.unique():
    cluster_indices = cluster_labels[cluster_labels == c].index
    cluster_truth_labels = [datasets[i].label for i in cluster_indices]

    print(f"{c}: {cluster_truth_labels}")

adjusted_rand_score(
    labels_true=[d.label for d in datasets],
    labels_pred=labels
), rand_score(
    labels_true=[d.label for d in datasets],
    labels_pred=labels  
)

# Repeated

In [ ]:
dynamics = []

dynamics.append(lambda x: np.sin(np.random.choice(np.linspace(0.1, 2, 20))*x[-1]))

dynamics.append(
    lambda x: np.exp(-(x[-1] - 1)**2),
)

dynamics.append(
    lambda x: np.exp(-(x[-1])**2),
)

In [ ]:
epsilon = 1
n_sets = 20
lag = 1

datasets = []

for d_id, fd in enumerate(dynamics):
    for i in range(n_sets):
        x0 = np.random.random()*2
        d = generate_time_series(x0, fd, 300, epsilon=epsilon)
        datasets.append(
            DataHolder(
                data=d,
                label=d_id
            )
        )

In [ ]:
# gridsearch = GridSearch(
#     KRRWrapper,
#     param_grid={
#         "bandwidth": np.linspace(0.1, 4, 40),
#         "reg": 1/10**np.linspace(1, 12, 20)
#     },
#     scoring=mean_squared_error
# )

paramsearch = OptunaHyperparameterTuning(
    KRRWrapper,
    param_grid={
        "bandwidth": dict(type="float", min=0, max=10),
        "reg": dict(type="float", min=1e-12, max=1e-3)
    },
    scoring=mean_squared_error
)

In [ ]:
results = dict(
    rand_score = [],
    adj_rand_score = [],
    silhouette_score_distance = [],
    silhouette_score_clustering = []
)


In [ ]:
# Loop
for i in tqdm.tqdm(range(1000)):
    datasets = []

    for d_id, fd in enumerate(dynamics):
        for i in range(n_sets):
            # x0 = np.random.random()
            d = generate_time_series(x0, fd, 1000)
            datasets.append(
                DataHolder(
                    data=d,#(d - d.min())/(d.max()- d.min()),
                    label=d_id
                )
            )

    for d in datasets:
        paramsearch.fit(d.data[:-1], d.data[1:])
        bandwidth = paramsearch.best_params["bandwidth"]
        reg = paramsearch.best_params["reg"]
        model = KernelRidgeRegression(
            kernels=[GaussianKernel(bandwidth=bandwidth)],
            reg = reg
        )

        model.fit(d.data)
        
        d.update(
            bandwidth=bandwidth,
            model=model,
            kernel=model.kernels[0],
            alpha=model.alpha
        )

    # Clustering
    similarity_matrix = np.zeros((len(datasets), len(datasets)))
    distance_matrix = np.zeros((len(datasets), len(datasets)))

    kernel = GaussianKernel(bandwidth=1)

    for i in range(len(datasets)):
        dii = (datasets[i].alpha.T@kernel(datasets[i].data[:-1]/datasets[i].bandwidth, datasets[i].data[:-1]/datasets[i].bandwidth)@datasets[i].alpha)[0][0]
        for j in range(i, len(datasets)):
            djj = (datasets[j].alpha.T@kernel(datasets[j].data[:-1]/datasets[j].bandwidth, datasets[j].data[:-1]/datasets[j].bandwidth)@datasets[j].alpha)[0][0]

            dij = (datasets[i].alpha.T@kernel(datasets[i].data[:-1]/datasets[i].bandwidth, datasets[j].data[:-1]/datasets[j].bandwidth)@datasets[j].alpha)[0][0]

            similarity_matrix[i][j] = dij
            distance_matrix[i][j] = dii + djj - 2*dij
            # similarity_matrix[i][j] = dij/np.sqrt(dii*djj) # Cosine distance
            similarity_matrix[j][i] = similarity_matrix[i][j]
            distance_matrix[j][i] = distance_matrix[i][j]

    # Graph laplacian
    D = np.diag(similarity_matrix.sum(axis=0))

    L = D - similarity_matrix

    evals, evecs = np.linalg.eig(L)
    eval_idx = np.argsort(evals)

    evals = evals[eval_idx]
    evecs = evecs[:, eval_idx]

    kmeans = KMeans(n_clusters=3)
    labels = kmeans.fit_predict(evecs[:, :4])

    results["rand_score"].append(rand_score(
        labels_true=[d.label for d in datasets],
        labels_pred=labels  
    ))

    results["adj_rand_score"].append(adjusted_rand_score(
        labels_true=[d.label for d in datasets],
        labels_pred=labels  
    ))

    results["silhouette_score_distance"].append(silhouette_score(
        distance_matrix,
        labels=[d.label for d in datasets],
        metric="precomputed"  
    ))

    results["silhouette_score_clustering"].append(silhouette_score(
        distance_matrix,
        labels=labels,
        metric="precomputed"  
    ))

    pd.DataFrame(results).to_csv("results.csv")